# Week 3 — Experiment Tracking & Model Versioning (MLflow)

**Learning Objectives (Week 3 – Experiment Tracking & Versioning)**  
- Track experiments, parameters, metrics, and artifacts with MLflow.  
- Register models and manage versions.  
- Enforce reproducibility for data references and code.

## Exercises
1. Configure MLflow tracking URI (local or remote).  
2. Log parameters, metrics, artifacts, and models.  
3. Compare runs and select a candidate model.  
4. Register the model; tag it with data and code versions.  
5. Export a run report to `artifacts/week3/`.

## Peer Validation
- **Peer Review Checklist:**  
  - Run metadata complete (params, metrics, artifacts).  
  - Reproducible code + data version reference.  
  - Registered model with meaningful tags.

In [ ]:
# %pip install mlflow scikit-learn pandas --quiet
import os, json, mlflow, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

os.makedirs("artifacts/week3", exist_ok=True)

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "file:artifacts/mlruns")
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("zap_week3_experiments")

# Load features produced in Week 2 (or synthesize if missing)
csv_path = "artifacts/week2/features.csv"
if not os.path.exists(csv_path):
    # fallback synthetic
    n = 3000
    df = pd.DataFrame({
        "amount": np.random.gamma(2.0, 50.0, n),
        "country_ES": np.random.randint(0,2,n),
        "country_FR": np.random.randint(0,2,n),
        "country_DE": np.random.randint(0,2,n),
        "channel_mobile": np.random.randint(0,2,n),
        "channel_store": np.random.randint(0,2,n),
        "label": np.random.choice([0,1], size=n, p=[0.7,0.3])
    })
else:
    df = pd.read_csv(csv_path)

X = df.drop(columns=["label"])
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

with mlflow.start_run(run_name="logreg_baseline") as run:
    params = {"C":1.0, "penalty":"l2", "solver":"lbfgs", "max_iter":200}
    mlflow.log_params(params)
    clf = LogisticRegression(**params)
    clf.fit(X_train, y_train)
    preds = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, preds)
    mlflow.log_metric("roc_auc", float(auc))
    # Log artifacts
    os.makedirs("artifacts/week3", exist_ok=True)
    X_test.sample(5, random_state=42).to_csv("artifacts/week3/sample_inputs.csv", index=False)
    mlflow.log_artifact("artifacts/week3/sample_inputs.csv")
    # Log model
    mlflow.sklearn.log_model(clf, "model", registered_model_name="zap_logreg_example")
    print("Run AUC:", auc)
    print("Run ID:", run.info.run_id)